In [2]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [1]:
! pip install datasets transformers[sentencepiece] sacrebleu sacremoses


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en"
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"

In [6]:
from datasets import load_dataset
from sklearn.metrics import precision_recall_fscore_support
from evaluate import load

#raw_datasets = load_dataset("opus100", "en-ur")
raw_datasets = load_dataset('json', data_files={
    'train': [r'C:\Users\firaa\OneDrive\Desktop\Urdu_English_Translation\combined_train.json'],
    #'validation':['/content/drive/MyDrive/Colab Notebooks/Tanzil/Tanzil_val_formatted.json'],
    'test': [r'C:\Users\firaa\OneDrive\Desktop\Urdu_English_Translation\combined_test.json']})


In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 46000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3637
    })
})

In [8]:
raw_datasets['test'][0]

{'translation': {'en': 'Even if a whiff of the Lord\'s chastisement were to touch them they would surely say: "Ah woe, alas, we were surely sinful."',
  'ur': 'حالانکہ انہیں عذاب الٰہی کی ہوا بھی چھوجائے تو کہہ اُٹھیں گے کہ افسوس ہم واقعی ظالم تھے'}}

In [9]:
metric1 = load("sacrebleu") # high scores are better
metric2 = load("ter")  # high score are worse
metric3 = load("chrf")
metric4 = load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\firaa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\firaa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\firaa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(raw_datasets["test"])

,translation
0,"{'en': 'Punjab chief minister Amarinder Singh had appealed to the Pakistan PM to immediately intervene that the devotees stranded in Gurdwara Nankana Sahib are rescued and the historic gurdwara is saved from the angry mob surrounding it', 'ur': 'پنجاب کے وزیراعلیٰ کیپٹن امریندر سنگھ نے پاکستان کے وزیر اعظم عمران خان سے فوراً اس معاملے میں مداخلت کرنے اور گرودوارہ میں پھنسے سکھوں کو محفوظ باہر نکالنے کی گزارش کی۔'}"
1,"{'en': 'We make their deeds attractive to those who do not believe in the Hereafter, so that they may wander in perplexity.', 'ur': 'بیشک جو لوگ آخرت پر ایمان نہیں رکھتے ہیں ہم نے ان کے اعمال کو ان کے لئے آراستہ کردیا ہے اور وہ ان ہی اعمال میں بھٹک رہے ہیں'}"
2,"{'en': 'From October onwards, SNA has been planning to virtual festivals of music, dance, drama, folk & tribal and puppetry besides the festivals in physical from January, 2021', 'ur': 'اکتوبر سے ایس این اے موسیقی ، رقص ، ڈرامہ، علاقائی اورقبائلی فن اورکٹھ پتلی کاورچوول تہوارپیش کرنے کا منصوبہ بنارہاہے ۔ اس کے علاوہ اس کاارادہ جنوری ، 2021سے ان تہواروں کو طبعی شکل میں پیش کرنے کا ہے ۔'}"
3,"{'en': 'Who shun all frivolities,', 'ur': 'اور لغو باتوں سے اعراض کرنے والے ہیں'}"
4,"{'en': 'Border Security Force (BSF)', 'ur': 'سرحدی حفاظتی دستہ (بی ایس ایف)'}"


In [12]:
from transformers import MarianTokenizer, MarianModel
    
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)

In [13]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-ur-en', vocab_size=62025, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	62024: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [14]:
from transformers import MarianMTModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokens = tokenizer.encode('This is a input.', return_tensors='pt')
print("These are tokens!", tokens)
for token in tokens[0]:
    print("This are decoded tokens!", tokenizer.decode([token]))

model = MarianMTModel.from_pretrained(model_checkpoint)
print(model.model.config)
#for e in model.embeddings.word_embeddings(tokens)[0]:
#    print("This is an embedding!", e)

These are tokens! tensor([[ 4117, 15061,    66,    22,    25,    73, 57904,   678,     5,     0]])
This are decoded tokens! T
This are decoded tokens! hi
This are decoded tokens! s
This are decoded tokens! is
This are decoded tokens! a
This are decoded tokens! 
This are decoded tokens! inpu
This are decoded tokens! t
This are decoded tokens! .
This are decoded tokens! </s>
MarianConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "Helsinki-NLP/opus-mt-ur-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62024,
  "decoder_vocab_size": 62025,
  

In [15]:
from transformers import DistilBertTokenizerFast, DistilBertModel

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
tokens = tokenizer.encode('This is a input.', return_tensors='pt')
print("These are tokens!", tokens)
for token in tokens[0]:
    print("This are decoded tokens!", tokenizer.decode([token]))

model = DistilBertModel.from_pretrained("distilbert-base-uncased")
print(model)
#for e in model.embeddings.word_embeddings(tokens)[0]:
#    print("This is an embedding!", e)

These are tokens! tensor([[ 101, 2023, 2003, 1037, 7953, 1012,  102]])
This are decoded tokens! [CLS]
This are decoded tokens! this
This are decoded tokens! is
This are decoded tokens! a
This are decoded tokens! input
This are decoded tokens! .
This are decoded tokens! [SEP]
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_

In [16]:
urdu_example = '''اے حبیب مکرم ! ہم آپ کو خود ایسا پڑھائیں گے کہ آپ کبھی نہیں بھولیں گے ۔'''
urdu_tokens = tokenizer.tokenize(urdu_example)
urdu_tokens

['ا',
 '##ے',
 'ح',
 '##ب',
 '##ی',
 '##ب',
 'م',
 '##ک',
 '##ر',
 '##م',
 '!',
 'ہ',
 '##م',
 'ا',
 '##پ',
 'ک',
 '##و',
 'خ',
 '##و',
 '##د',
 'ا',
 '##ی',
 '##س',
 '##ا',
 '[UNK]',
 'گ',
 '##ے',
 'ک',
 '##ہ',
 'ا',
 '##پ',
 'ک',
 '##ب',
 '##ھ',
 '##ی',
 'ن',
 '##ہ',
 '##ی',
 '##ں',
 'ب',
 '##ھ',
 '##و',
 '##ل',
 '##ی',
 '##ں',
 'گ',
 '##ے',
 '[UNK]']

In [17]:
eng_example ="(O Esteemed Beloved!) We shall teach you (in a way) that nothing shall you (ever) forget."
eng_tokens = tokenizer.tokenize(eng_example)
eng_tokens

['(',
 'o',
 'esteem',
 '##ed',
 'beloved',
 '!',
 ')',
 'we',
 'shall',
 'teach',
 'you',
 '(',
 'in',
 'a',
 'way',
 ')',
 'that',
 'nothing',
 'shall',
 'you',
 '(',
 'ever',
 ')',
 'forget',
 '.']

In [18]:
max_input_length = 128
max_target_length = 128
source_lang = "ur"
target_lang = "en"
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
#preprocess_function(raw_datasets['train'][:2])
preprocess_function(raw_datasets['test'][:2])

c:\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[101, 1276, 25573, 23673, 18511, 29841, 29845, 1270, 15915, 29845, 24830, 29843, 1288, 29822, 25573, 29816, 1270, 23673, 29845, 24830, 1304, 24830, 1308, 29836, 25573, 1271, 29844, 24830, 1303, 29844, 29836, 29819, 25573, 14498, 29846, 1273, 29836, 1304, 29845, 29845, 1270, 29838, 29844, 24830, 29843, 1305, 29846, 1304, 29845, 1270, 29833, 29824, 29836, 29824, 1308, 22192, 1298, 25573, 29834, 29830, 24830, 1287, 25573, 23673, 22192, 1273, 29844, 29846, 102], [101, 1270, 29836, 17149, 1308, 22192, 1292, 24830, 25573, 22192, 29817, 1304, 29846, 1278, 15915, 1270, 15915, 29826, 25573, 29833, 1304, 24830, 1273, 17149, 25573, 29823, 29836, 1292, 25573, 14498, 22192, 1304, 17149, 24830, 29843, 1305, 29846, 1270, 29836, 17149, 1304, 29824, 24830, 1296, 29833, 29824, 1302, 17149, 1270, 15394, 15915, 24830, 1287, 23673, 22192, 1296, 29845, 24830, 29843, 1304, 24830, 25573, 1275, 25573, 14498, 29846, 1305, 25573, 1270, 29836, 17149, 1304, 29824, 24830, 1304, 25573, 1288, 22192, 23

In [20]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [31]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(30000))
#small_valid_dataset = tokenized_datasets["validation"]#[0:1500] #.shuffle(seed=42).select(range(2000))
small_test_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import MarianMTModel, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = MarianMTModel.from_pretrained(model_checkpoint)

In [23]:
model.config.activation_dropout = 0.01
model.config.add_final_layer_norm = True
model.config.attention_dropout = 0.01
model.config.add_cross_attention = True
model.config.decoder_attention_heads = 10

In [24]:
model.config

MarianConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "Helsinki-NLP/opus-mt-ur-en",
  "activation_dropout": 0.01,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_cross_attention": true,
  "add_final_layer_norm": true,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.01,
  "bad_words_ids": [
    [
      62024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 10,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62024,
  "decoder_vocab_size": 62025,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 62025,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "la

In [25]:
#model.config.early_stopping 

In [26]:
import accelerate

accelerate.__version__

'1.4.0'

In [27]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"finetuned-marian-mt-{source_lang}-{target_lang}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch", 
    learning_rate=2e-5,
    optim = "adamw_torch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.001,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    generation_num_beams=6,
    load_best_model_at_end= True,
)

c:\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [29]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    s_bleu_result = metric1.compute(predictions=decoded_preds, references=decoded_labels)
    #s_bleu_result = {k: round(v, 2) for k, v in s_bleu_result.items()}

    ter_result = metric2.compute(predictions=decoded_preds, references=decoded_labels, ignore_punct=True, case_sensitive=False)
    #ter_result = {k: round(v, 2) for k, v in ter_result.items()}

    chrf = metric3.compute(predictions=decoded_preds, references=decoded_labels)
    #chrf = {k: round(v, 2) for k, v in chrf.items()}
    
    meteor = metric4.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = round(meteor['meteor'], 2)

    #google_bleu_score = metric5.compute(predictions=decoded_preds, references=decoded_labels)
    #google_bleu_result = round(google_bleu_score['google_bleu'], 2)
    #preds = pred.predictions.argmax(-1)
    #a_preds = np.argmax(preds, axis=-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, a_preds)
    #accuracy = accuracy_score(labels, a_preds)

    
    #accuracy = metric4.compute(predictions=a_preds, references=labels)
    #precision = metric5.compute(predictions=a_preds, references=labels)
    #recall = metric6.compute(predictions=a_preds, references=labels)

    result = {"bleu": round(s_bleu_result["score"],2), "chrF": round(chrf["score"],2) ,"TER": round(ter_result["score"],2), 
              "METEOR": meteor_result}#,"Google Bleu": google_bleu_result["google_bleu"]}#,"recall": bert_score["recall"],"f1": bert_score["f1"]}
    #prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    #result["gen_len"] = np.mean(prediction_lens)
    
    return result

In [33]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\firaa\AppData\Local\Temp\ipykernel_19448\1904768668.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained")

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained
Configuration saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-tanzil-trained/special_tokens_map.json


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 16


{'epoch': 20.0,
 'eval_METEOR': 0.37,
 'eval_TER': 82.47,
 'eval_bleu': 14.68,
 'eval_chrF': 41.45,
 'eval_loss': 0.18391641974449158,
 'eval_runtime': 188.4539,
 'eval_samples_per_second': 7.96,
 'eval_steps_per_second': 0.499}

In [ ]:
model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-pib-trained"
#model_checkpoint ="Helsinki-NLP/opus-mt-ur-en"
from transformers import MarianTokenizer, MarianMTModel
    
tokenizer = MarianTokenizer.from_pretrained(model_checkpoint)
model = MarianMTModel.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
model.config

MarianConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/finetuned-marian-mt-ur-en-pib-trained",
  "activation_dropout": 0.01,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_cross_attention": true,
  "add_final_layer_norm": true,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.01,
  "bad_words_ids": [
    [
      62024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62024,
  "decoder_vocab_size": 62025,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "extra_pos_embeddings": 62025,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": tr

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2100
  Batch size = 16


{'eval_loss': 5.186152458190918,
 'eval_bleu': 8.13,
 'eval_chrF': 32.43,
 'eval_TER': 89.31,
 'eval_METEOR': 0.28,
 'eval_runtime': 558.2769,
 'eval_samples_per_second': 3.762,
 'eval_steps_per_second': 0.236}

In [ ]:
trainer.compute_loss_context_manager

<bound method Trainer.compute_loss_context_manager of <transformers.trainer_seq2seq.Seq2SeqTrainer object at 0x7f6f93dc6a90>>

In [ ]:
from transformers import pipeline

translator = pipeline("translation_ur_to_en", model=model_checkpoint)

input = "آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت "
ground = "PM condoles loss of lives in an accident in Andhra Pradesh"
output = translator(input)
print("Input:  " + input + "\n")
print("Output:  " + output[0]['translation_text'] + "\n")
print("Ground:  " + ground + "\n")

Input:  آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت 

Output:  PM condoles Mahotssamen in an accident accident in Andhra Pradesh

Ground:  PM condoles loss of lives in an accident in Andhra Pradesh



In [ ]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"
translator = pipeline("translation_ur_to_en", model=model_checkpoint)

input = "آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت "
ground = "PM condoles loss of lives in an accident in Andhra Pradesh"
output = translator(input)
print("Input:  " + input + "\n")
print("Output:  " + output[0]['translation_text'] + "\n")
print("Ground:  " + ground + "\n")

Input:  آندھراپردیش میں ہوئے ایک حادثے میں مہلوکین کیلئے وزیراعظم کی تعزیت 

Output:  In an accident in São Paulo, at the time of a accident, the prime minister for the prime minister in São Paulo.

Ground:  PM condoles loss of lives in an accident in Andhra Pradesh

